In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt


from keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, Multiply , Embedding
from keras.layers import RepeatVector, Dense, Activation, Lambda
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.models import load_model, Model
import keras.backend as K

from faker import Faker
import random
from tqdm import tqdm
from babel.dates import format_date
import matplotlib.pyplot as plt
%matplotlib inline



Using TensorFlow backend.


In [2]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [3]:
os.getcwd()    

'/home/varun/Desktop/speechReco'

In [4]:
ls


 0/                                        libredataset.ipynb
 0.zip                                     lines/
 1835_3176_compressed_glove.6B.200d.txt/   spectogram.py
 Attention_Model.ipynb                     speechrecognaization.ipynb
 data_generator.ipynb                      test-clean/
'demo (1).ipynb'                           Untitled1.ipynb
 en.txt                                    Untitled.ipynb
 final.ipynb


In [5]:
def create_word_embeddings(file_path):
    
    with open(file_path , 'r') as f:
        wordToEmbedding = {}
        wordToIndex = {}
        indexToWord = {}
        
        count = 0
        for line in f:
            data = line.strip().split()
            token = data[0]
            wordToEmbedding[token] = np.array(data[1:] ,dtype = np.float64)
            count += 1
            if count == 20000 :
                break

        
        tokens = sorted(wordToEmbedding.keys())
        for idx , token in enumerate(tokens):
            idx = idx + 1 #for zero masking
            wordToIndex[token] = idx
            indexToWord[idx] = token

    return wordToEmbedding , wordToIndex , indexToWord
            
            

In [6]:
wordToEmbedding , wordToIndex , indexToWord = create_word_embeddings(os.getcwd()+ '/1835_3176_compressed_glove.6B.200d.txt/glove.6B.200d.txt')

In [7]:
print("size: " ,len(wordToIndex))

size:  20000


In [8]:
def create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord):
    
    vocablen = len(wordToIndex) + 1 #for zero masking
    embedding_dimensions = 200
    
    embeddingMatrix = np.zeros((vocablen , embedding_dimensions))
    
    for word , index in wordToIndex.items():
        embeddingMatrix[index , :] = wordToEmbedding[word]
        
    embeddingLayer = Embedding(vocablen , embedding_dimensions , weights = [embeddingMatrix] , trainable = False)
    
    return embeddingLayer

In [9]:
embeddingLayer = create_pretrained_embedding_layer(wordToEmbedding , wordToIndex , indexToWord)

In [10]:
def softmax(x, axis=1):
    """Softmax activation function.
    # Arguments
        x : Tensor.
        axis: Integer, axis along which the softmax normalization is applied.
    # Returns
        Tensor, output of softmax transformation.
    # Raises
        ValueError: In case `dim(x) == 1`.
    """
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')

In [11]:
Tx = 10
repeator = RepeatVector(Tx)
concatenator = Concatenate(axis=-1)
densor1 = Dense(10, activation = "tanh")
densor2 = Dense(1, activation = "relu")
activator = Activation(softmax, name='attention_weights') # We are using a custom softmax(axis = 1) loaded in this notebook
dotor = Dot(axes = 1)


In [12]:
def data_generator(file_path, wordToIndex, eos_token, batch_size = 50, max_len = 30):
    start = 0
    end = batch_size * 2
    
    def custom_tokenizer(line , max_len ,wordToIndex):
        data = []
        for word in line.lower().split():
            if word not in wordToIndex:
                data.append(0)
            else :
                data.append(wordToIndex[word])
#         data.append(eos_token)
        while len(data) < max_len:
            data.append(0)
        if len(data) > max_len:
            data = data[:max_len]
        return np.array(data)
        
            
    while True:
        contexts = np.zeros((batch_size , max_len))
        responses =  np.zeros((batch_size , max_len))
        index = 0
        with open(file_path ,'r') as txtfile:
            lines = txtfile.readlines()
            check = len(lines)
            if start >= check - batch_size :
                start = 0
                end = batch_size * 2
            for idx in range(start,end,2):
                contexts[index , :] = custom_tokenizer(lines[idx] , max_len , wordToIndex)
                responses[index, :] = custom_tokenizer(lines[idx+1] , max_len , wordToIndex)
                index +=1
                
#             txtfile.close()
        start = end
        end = end + 2*batch_size
        s0 = np.zeros((batch_size, 128))
        c0 = np.zeros((batch_size, 128))
    
        yield ([contexts,s0,c0] , list(responses.swapaxes(0,1)))                

In [13]:
a =data_generator(os.getcwd()+ '/lines/lines-aaa' , wordToIndex,len(wordToIndex)+1,300,30)

In [14]:
next(a)

([array([[14039.,  3362.,  9289., ...,     0.,     0.,     0.],
         [ 9373.,  2069., 19691., ...,     0.,     0.,     0.],
         [12169.,     0.,  8193., ...,     0.,     0.,     0.],
         ...,
         [    0.,  8252., 12795., ...,     0.,     0.,     0.],
         [    0.,     0.,     0., ...,     0.,     0.,     0.],
         [   32., 19561.,     0., ...,     0.,     0.,     0.]]),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]),
  array([[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]])],
 [array([14179.,     0.,  6937.,     0.,     0., 19331.,     0.,     0.,
             0

In [15]:
def one_step_attention(a, s_prev):
    """
    Performs one step of attention: Outputs a context vector computed as a dot product of the attention weights
    "alphas" and the hidden states "a" of the Bi-LSTM.
    
    Arguments:
    a -- hidden state output of the Bi-LSTM, numpy-array of shape (m, Tx, 2*n_a)
    s_prev -- previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, n_s)
    
    Returns:
    context -- context vector, input of the next (post-attention) LSTM cell
    """
    
    ### START CODE HERE ###
    # Use repeator to repeat s_prev to be of shape (m, Tx, n_s) so that you can concatenate it with all hidden states "a" (≈ 1 line)
    s_prev = repeator(s_prev)
    # Use concatenator to concatenate a and s_prev on the last axis (≈ 1 line)
    # For grading purposes, please list 'a' first and 's_prev' second, in this order.
    concat = concatenator(inputs =([a,s_prev]))
    # Use densor1 to propagate concat through a small fully-connected neural network to compute the "intermediate energies" variable e. (≈1 lines)
    e = densor1(concat)
    # Use densor2 to propagate e through a small fully-connected neural network to compute the "energies" variable energies. (≈1 lines)
    energies = densor2(e)
    # Use "activator" on "energies" to compute the attention weights "alphas" (≈ 1 line)
    alphas = activator(energies)
    # Use dotor together with "alphas" and "a" to compute the context vector to be given to the next (post-attention) LSTM-cell (≈ 1 line)
    context = dotor([alphas,a])
    ### END CODE HERE ###
    
    return context


In [16]:
post_activation_LSTM_cell = LSTM(128, return_state = True) # post-attention LSTM 
output_layer = Dense(20000, activation=softmax)

In [17]:

def model(Tx, Ty, n_a, n_s, human_vocab_size, machine_vocab_size):

    
    # Define the inputs of your model with a shape (Tx,)
    # Define s0 (initial hidden state) and c0 (initial cell state)
    # for the decoder LSTM with shape (n_s,)
    X = Input(shape=(Tx,))
    s0 = Input(shape=(n_s,), name='s0')
    c0 = Input(shape=(n_s,), name='c0')
    s = s0
    c = c0
    
    # Initialize empty list of outputs
    outputs = []
    
    ### START CODE HERE ###
    
    # Step 1: Define your pre-attention Bi-LSTM. (≈ 1 line)
    e = embeddingLayer(X)
    a = Bidirectional(LSTM(units = n_a ,return_sequences = True))(e)
    
    # Step 2: Iterate for Ty steps
    for t in range(Ty):
    
        # Step 2.A: Perform one step of the attention mechanism to get back the context vector at step t (≈ 1 line)
        context = one_step_attention(a, s)
        
        # Step 2.B: Apply the post-attention LSTM cell to the "context" vector.
        # Don't forget to pass: initial_state = [hidden state, cell state] (≈ 1 line)
        s, _, c = post_activation_LSTM_cell(inputs = context,initial_state = [s,c])
        
        # Step 2.C: Apply Dense layer to the hidden state output of the post-attention LSTM (≈ 1 line)
        out = output_layer(s)
        
        # Step 2.D: Append "out" to the "outputs" list (≈ 1 line)
        outputs.append(out)
    
    # Step 3: Create model instance taking three inputs and returning the list of outputs. (≈ 1 line)
    model = Model(inputs = [X,s0,c0],outputs = outputs)
    
    ### END CODE HERE ###
    
    return model

In [18]:
model = model(10, 10, 128, 128, 20000, 20000)


In [19]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 10, 200)      4000200     input_1[0][0]                    
__________________________________________________________________________________________________
s0 (InputLayer)                 (None, 128)          0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 10, 256)      336896      embedding_1[0][0]                
____________________________________________________________________________________________

In [20]:
opt = Adam(lr = 0.005, beta_1 = 0.9,beta_2 = 0.999 , decay = 0.01)
model.compile(optimizer = opt , loss = "sparse_categorical_crossentropy" ,metrics = ['accuracy'])

In [21]:
s0 = np.zeros((50, 128))
c0 = np.zeros((50, 128))

In [22]:
model.fit(data_generator(os.getcwd()+ '/lines/lines-aaa' , wordToIndex,len(wordToIndex)+1,300,10),steps_per_epoch=50  , epochs =2 )

Epoch 1/2
50/50 [==============================] - 48s 960ms/step - loss: 40.1596 - dense_3_loss: 2.3050 - dense_3_accuracy: 0.3331 - dense_3_accuracy_1: 0.3849 - dense_3_accuracy_2: 0.4489 - dense_3_accuracy_3: 0.5450 - dense_3_accuracy_4: 0.6416 - dense_3_accuracy_5: 0.7226 - dense_3_accuracy_6: 0.7845 - dense_3_accuracy_7: 0.8339 - dense_3_accuracy_8: 0.8713 - dense_3_accuracy_9: 0.9015
Epoch 2/2
50/50 [==============================] - 42s 850ms/step - loss: 27.9325 - dense_3_loss: 1.0053 - dense_3_accuracy: 0.3120 - dense_3_accuracy_1: 0.3485 - dense_3_accuracy_2: 0.4171 - dense_3_accuracy_3: 0.5307 - dense_3_accuracy_4: 0.6327 - dense_3_accuracy_5: 0.7343 - dense_3_accuracy_6: 0.7995 - dense_3_accuracy_7: 0.8509 - dense_3_accuracy_8: 0.8873 - dense_3_accuracy_9: 0.9150


In [23]:
import math
def beam_search_decoder(predictions, top_k = 3):

    output_sequences = [([], 0)]
    
    for token_probs in predictions:
        new_sequences = []
        token_probs = token_probs.reshape(20000 ,)
        for old_seq, old_score in output_sequences:
            for char_index in range(len(token_probs)):
                new_seq = old_seq + [char_index]
                new_score = old_score + math.log(token_probs[char_index])
                new_sequences.append((new_seq, new_score))
                
        output_sequences = sorted(new_sequences, key = lambda val: val[1], reverse = True)
        output_sequences = output_sequences[:top_k]
        
    return output_sequences

In [24]:
def custom_tokenizer(line , max_len ,wordToIndex):
        data = []
        for word in line.lower().split():
            if word not in wordToIndex:
                data.append(0)
            else :
                data.append(wordToIndex[word])
#         data.append(eos_token)
        while len(data) < max_len:
            data.append(0)
        if len(data) > max_len:
            data = data[:max_len]
        return np.array(data)

s = " How are you ?"

s = custom_tokenizer(s , 10 ,wordToIndex)
s = s.reshape(1,10)
s.shape

(1, 10)

In [25]:
predictions = model.predict([s,np.zeros((1,128)),np.zeros((1,128))])

In [26]:
out = beam_search_decoder(predictions, top_k = 3)

In [27]:
predictions  = list(predictions)

In [28]:
# t = predictions[1].reshape(10000,)

In [30]:
# t.shape

In [31]:
out

[([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], -5.672516718936462),
 ([18090, 0, 0, 0, 0, 0, 0, 0, 0, 0], -8.086307466720754),
 ([18264, 0, 0, 0, 0, 0, 0, 0, 0, 0], -8.282365000807149)]

In [32]:
predictions

[array([[2.5905043e-01, 1.3340020e-04, 4.1940680e-04, ..., 4.1368625e-07,
         3.6008859e-07, 3.4735356e-07]], dtype=float32),
 array([[5.7170045e-01, 3.7553433e-05, 1.6187096e-04, ..., 2.7899050e-08,
         2.3607628e-08, 2.2554760e-08]], dtype=float32),
 array([[6.1882669e-01, 2.9481080e-05, 1.3360092e-04, ..., 1.7715690e-08,
         1.4945449e-08, 1.4254193e-08]], dtype=float32),
 array([[6.2486821e-01, 2.8509627e-05, 1.3005186e-04, ..., 1.6617507e-08,
         1.4009768e-08, 1.3359242e-08]], dtype=float32),
 array([[6.2568170e-01, 2.8380073e-05, 1.2957613e-04, ..., 1.6473512e-08,
         1.3887125e-08, 1.3241940e-08]], dtype=float32),
 array([[6.2579161e-01, 2.8362572e-05, 1.2951191e-04, ..., 1.6454109e-08,
         1.3870582e-08, 1.3226090e-08]], dtype=float32),
 array([[6.2580609e-01, 2.8360253e-05, 1.2950330e-04, ..., 1.6451509e-08,
         1.3868390e-08, 1.3224001e-08]], dtype=float32),
 array([[6.2580824e-01, 2.8359918e-05, 1.2950202e-04, ..., 1.6451125e-08,
         

In [33]:
indexToWord[18090]

'the'

In [34]:
indexToWord[18264]

'to'